In [1]:
from cot import Collection

In [2]:
# collection = Collection.from_json("./thoughtsource_data/thoughtsource_1_empty.json")

In [3]:
collection = Collection(["worldtree"], verbose=False)
collection = collection.select(number_samples=1)

Loading worldtree...


In [4]:
collection

| Name      |   Train | Valid   | Test   |
|-----------|---------|---------|--------|
| worldtree |       1 | -       | -      |

Not loaded: ['aqua', 'asdiv', 'commonsense_qa', 'entailment_bank', 'gsm8k', 'mawps', 'med_qa', 'medmc_qa', 'open_book_qa', 'pubmed_qa', 'qed', 'strategy_qa', 'svamp']

In [5]:

config={
    "cot_trigger_keys": ['kojima-01'],
    "answer_extraction_keys": ['kojima-A-D'], 
    "author" : "konstantin",
    "temperature": 0,
    "max_tokens": 512,
    "verbose": False,
    "warn": False,
}

In [69]:
collection.generate(config=config)

Generating worldtree...


In [82]:
collection.evaluate()
collection.full_text_prompts()
collection.dump("foo.json")

Evaluating worldtree...


In [77]:
api_model = [("openai", "text-davinci-003"), ("cohere", "command-xlarge-nightly")]

In [78]:
for api, model in api_model:
    config["api_service"] = api
    config["engine"] = model
    collection.generate(config=config)

Generating worldtree...
Generating worldtree...


In [6]:
from langchain import OpenAI, LLMChain, Prompt

In [7]:
input = "A parent and a child share several characteristics. Both individuals are tall, have curly hair, are good cooks, and have freckles. Which of these characteristics is a learned behavior?\nA) being tall\nB) having curly hair\nC) being a good cook\nD) having freckles\n\nAnswer: Let's think step by step."

In [8]:
template = "{prompt}"
prompt = Prompt(template=template, input_variables=["prompt"])

In [9]:
engine = "text-davinci-003"
max_tokens = 512
temperature = 0.0

In [10]:
llm=OpenAI(
    # parameter options: https://beta.openai.com/docs/api-reference/completions/create-completion
    model_name=engine,
    max_tokens=max_tokens,
    temperature=temperature,
    # type: ignore (suppress pylance error)
)

In [11]:
chain = LLMChain(
    prompt=prompt,
    llm=llm
)

In [17]:
from langchain import LLMCheckerChain

In [ ]:
LLMCheckerChain

In [136]:
response = chain.predict(prompt=input, stop=None)
response

' Being tall and having curly hair are physical characteristics that are inherited, not learned. Having freckles is also a physical characteristic that is inherited. Therefore, the correct answer is C) being a good cook, which is a learned behavior.'

In [137]:
llm

OpenAI(cache=None, verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x7f93bbf55120>, client=<class 'openai.api_resources.completion.Completion'>, model_name='text-davinci-003', temperature=0.0, max_tokens=512, top_p=1, frequency_penalty=0, presence_penalty=0, n=1, best_of=1, model_kwargs={}, openai_api_key=None, batch_size=20, request_timeout=None, logit_bias={}, max_retries=6, streaming=False)

In [139]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

ValidationError: 2 validation errors for LLMChain
prompt
  Can't instantiate abstract class BasePromptTemplate with abstract methods _prompt_type, format (type=type_error)
llm
  Can't instantiate abstract class BaseLLM with abstract methods _agenerate, _generate, _llm_type (type=type_error)

In [37]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9)
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

In [38]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain only specifying the input variable.
print(chain.run("colorful socks"))



Splashy Sox.


In [39]:
second_prompt = PromptTemplate(
    input_variables=["company_name"],
    template="Write a catchphrase for the following company: {company_name}",
)
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [18]:
from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[chain, chain_two], verbose=True)

# Run the chain specifying only the input variable for the first chain.
catchphrase = overall_chain.run("colorful socks")
print(catchphrase)

ValidationError: 1 validation error for SimpleSequentialChain
return_all
  extra fields not permitted (type=value_error.extra)

In [19]:
from langchain.chains import SequentialChain
overall_chain = SequentialChain(chains=[chain, chain_two], verbose=True, return_all=True)

# Run the chain specifying only the input variable for the first chain.
catchphrase = overall_chain.run("colorful socks")
print(catchphrase)

KeyError: 'input_variables'

In [130]:
overall_chain.memory

In [115]:
catchphrase

'\n\n"Put a little color in your wardrobe - Get your colorful socks today!"'

In [42]:
from langchain.chains import LLMCheckerChain
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)

text = "What type of mammal lays the biggest eggs?"

checker_chain = LLMCheckerChain(llm=llm, verbose=True)

result = checker_chain.run(text)



> Entering new LLMCheckerChain chain...


> Entering new SequentialChain chain...

> Finished chain.

> Finished chain.


In [43]:
result

{'question': 'What type of mammal lays the biggest eggs?',
 'statement': '\nThe largest mammal that lays eggs is the platypus.',
 'assertions': '\n• Platypus is a mammal \n• Mammals can lay eggs \n• Platypus is the largest mammal that lays eggs',
 'checked_assertions': '\n• Platypus is a mammal: True\n• Mammals can lay eggs: False - Mammals are viviparous, meaning they give birth to live young.\n• Platypus is the largest mammal that lays eggs: False - The largest mammal that lays eggs is the echidna, which is a monotreme like the platypus.',
 'revised_statement': ' The echidna is the largest mammal that lays eggs.'}

In [41]:
result

' The platypus is the largest mammal that lays eggs.'

In [25]:
# This is an LLMChain to write a synopsis given a title of a play and the era it is set in.
llm = OpenAI(temperature=.7)
template = """You are a playwright. Given the title of play and the era it is set in, it is your job to write a synopsis for that title.

Title: {title}
Era: {era}
Playwright: This is a synopsis for the above play:"""
prompt_template = PromptTemplate(input_variables=["title", 'era'], template=template)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="synopsis")

In [26]:
# This is an LLMChain to write a review of a play given a synopsis.
llm = OpenAI(temperature=.7)
template = """You are a play critic from the New York Times. Given the synopsis of play, it is your job to write a review for that play.

Play Synopsis:
{synopsis}
Review from a New York Times play critic of the above play:"""
prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="review")

In [27]:
# This is the overall chain where we run these two chains in sequence.
from langchain.chains import SequentialChain
overall_chain = SequentialChain(
    chains=[synopsis_chain, review_chain],
    input_variables=["era", "title"],
    # Here we return multiple variables
    output_variables=["synopsis", "review"],
    verbose=True)

In [28]:
review = overall_chain({"title":"Tragedy at sunset on the beach", "era": "Victorian England"})



> Entering new SequentialChain chain...

> Finished chain.


In [29]:
review

{'title': 'Tragedy at sunset on the beach',
 'era': 'Victorian England',
 'synopsis': "\n\nTragedy at sunset on the beach is a tale of love, loss, and sorrow set in Victorian England. It follows the story of a young couple, John and Mary, who have been deeply in love since they first met. Despite their growing love, their families are from opposite sides of the social divide and forbid the two to be together. In a last-ditch effort to be together, John and Mary decide to elope and escape to the beach, where they plan to get married. \n\nOn their way to the beach, they are stopped by John's father, who has followed them in a desperate attempt to keep them apart. In a desperate moment, John's father shoots Mary, killing her instantly. John is left heartbroken and alone on the beach, surrounded by the setting sun. \n\nThis tragedy serves as a reminder of the power of social divide and the strength of true love. In the end, John is left to mourn his lost love while the audience is left to 